# Test the full connected layer model

## Import necessary modules

In [1]:
# python built-in libraries
import datetime

In [2]:
# choose the tensorflow log level
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

In [3]:
# import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# ohter packages
import numpy as np

import tfhelper.core as tfhelper

## Global Variables

In [5]:
# sampling
FEATURES_FILE = "./data/tensors/features.npy"  # path to the file of the feature tensor
LABELS_FILE = "./data/tensors/labels.npy"  # path to the file of the feature tensor
FRACTIONS = (0.8, 0.1, 0.1)  # train, validation, test
BATCH_SIZE = 256  # size of the batch
BUFFER_SIZE = BATCH_SIZE * 2  # size of the shuffle buffer
# training
LEARNING_RATE = 0.001  # starting learning rate
BETA1 = 0.9  # decay 1
BETA2 = 0.999  # decay 2
EPOCHS = 50  # number of epochs
# saving
TIME_STAMP = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # time stamp
SAVE_LOC = "./data/models/cnn_1_{}".format(TIME_STAMP)  # path to the folder to save the model
LOG_LOC = "./data/logs/fit/{}".format(TIME_STAMP)  # path to the log, if you change this, you also need to change it in the run_tensorboard

## Obtain data

The data is a 209 data point PDF. The label is a one-hot 2 dim vector. `10` means major phase >= threshold, `01` means major phase <= threshold.

In [6]:
def load_data():
    # load
    features, labels = np.load("./data/tensors/features.npy"), np.load("./data/tensors/labels.npy")
    # shuffle
    n = features.shape[0]
    shuffled_idx = np.random.permutation(n)
    features, labels = features[shuffled_idx], labels[shuffled_idx]
    # split
    f0, f1, f2 = FRACTIONS
    i, j, k = round(f0 * n), round((f0 + f1) * n), round((f0 + f1 + f2) * n)
    train_data = tf.data.Dataset.from_tensor_slices((features[:i], labels[:i])).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    valid_data = tf.data.Dataset.from_tensor_slices((features[i:j], labels[i:j])).batch(BATCH_SIZE)
    test_data = tf.data.Dataset.from_tensor_slices((features[j:k], labels[j:k])).batch(BATCH_SIZE)
    return train_data, valid_data, test_data

In [7]:
train_data, valid_data, test_data = load_data()

## Create the model

We use the logistric regression. It is a single layer with a softmax function.

In [8]:
def create_model():
    model = keras.Sequential()
    model.add(keras.Input(shape=(209,)))
    model.add(keras.layers.Reshape((209, 1)))
    model.add(keras.layers.Conv1D(16, 32, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.Conv1D(16, 32, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.MaxPooling1D(4))
    model.add(keras.layers.Conv1D(64, 16, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.Conv1D(64, 16, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.MaxPooling1D(4))
    model.add(keras.layers.Conv1D(256, 8, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.Conv1D(256, 8, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.Conv1D(512, 4, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.Conv1D(512, 4, strides=1, activation='relu', padding="same"))
    model.add(keras.layers.MaxPooling1D(2))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(2, activation='softmax'))
    return model

In [9]:
model = create_model()

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 209, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 209, 16)           528       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 209, 16)           8208      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 52, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 52, 64)            16448     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 52, 64)            65600     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 64)            0

## Choose optimization method

In [11]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(LEARNING_RATE, BETA1, BETA2),
    # Loss function to minimize
    loss=keras.losses.CategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[
        keras.metrics.CosineSimilarity(),
        keras.metrics.TopKCategoricalAccuracy(k=1)
    ],
)

## Train the model

In [12]:
# tensor board
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_LOC, histogram_freq=1)

In [13]:
history = model.fit(
    x=train_data,
    epochs=EPOCHS,
    callbacks=[tensorboard_callback],
    validation_data=valid_data
)

Epoch 1/50
344/344 [==============================] - 26s 36ms/step - loss: 0.6722 - cosine_similarity: 0.7287 - top_k_categorical_accuracy: 0.6354 - val_loss: 0.6575 - val_cosine_similarity: 0.7317 - val_top_k_categorical_accuracy: 0.6340
Epoch 2/50
344/344 [==============================] - 7s 21ms/step - loss: 0.6564 - cosine_similarity: 0.7323 - top_k_categorical_accuracy: 0.6355 - val_loss: 0.6573 - val_cosine_similarity: 0.7318 - val_top_k_categorical_accuracy: 0.6340
Epoch 3/50
344/344 [==============================] - 7s 21ms/step - loss: 0.6561 - cosine_similarity: 0.7325 - top_k_categorical_accuracy: 0.6358 - val_loss: 0.6570 - val_cosine_similarity: 0.7319 - val_top_k_categorical_accuracy: 0.6340
Epoch 4/50
344/344 [==============================] - 7s 21ms/step - loss: 0.6561 - cosine_similarity: 0.7326 - top_k_categorical_accuracy: 0.6356 - val_loss: 0.6569 - val_cosine_similarity: 0.7320 - val_top_k_categorical_accuracy: 0.6340
Epoch 5/50
344/344 [=======================

## Evaluate the model

In [14]:
result = model.evaluate(
    x=test_data,
    return_dict=True
)

43/43 [==============================] - 0s 9ms/step - loss: 0.6569 - cosine_similarity: 0.7320 - top_k_categorical_accuracy: 0.6338


In [15]:
for name, val in result.items():
    print("{:20s} {:.4f}".format(name, val))

loss                 0.6569
cosine_similarity    0.7320
top_k_categorical_accuracy 0.6338


## Save the model

In [16]:
model.save(SAVE_LOC)

INFO:tensorflow:Assets written to: ./data/models/cnn_1_20210912-230847/assets
